# Data Preprocessing and Migrating

This notebook loads and preprocesses data from CSV and JSON files into a MySQL database (`finance`). It populates the `users`, `mcc`, `cards` and `transactions`

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

# Configuration variables
MYSQL_HOST = 'localhost'
MYSQL_PORT = '30306'
MYSQL_DATABASE = 'finance'
MYSQL_USER = 'root'
MYSQL_PASSWORD = 'root123'

engine = create_engine(f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}')

In [2]:
# POPULATE USERS TABLE

# Preprocess monetary columns by removing '$' and converting to numeric
df = pd.read_csv('/home/bnguyen/Desktop/finance_analytics/datasets/users_data.csv')
df['per_capita_income'] = pd.to_numeric(df['per_capita_income'].astype(str).str.replace('$', ''), errors='coerce')
df['yearly_income'] = pd.to_numeric(df['yearly_income'].astype(str).str.replace('$', ''), errors='coerce')
df['total_debt'] = pd.to_numeric(df['total_debt'].astype(str).str.replace('$', ''), errors='coerce')

# collumn 'id' to 'client_id' to match the database
df.rename(columns={
    'id': 'client_id',
}, inplace=True)

df.to_sql('users', con=engine, if_exists='append', index=False, chunksize=60000)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '825' for key 'users.PRIMARY'")
[SQL: INSERT INTO users (client_id, current_age, retirement_age, birth_year, birth_month, gender, address, latitude, longitude, per_capita_income, yearly_income, total_debt, credit_score, num_credit_cards) VALUES (%(client_id)s, %(current_age)s, %(retirement_age)s, %(birth_year)s, %(birth_month)s, %(gender)s, %(address)s, %(latitude)s, %(longitude)s, %(per_capita_income)s, %(yearly_income)s, %(total_debt)s, %(credit_score)s, %(num_credit_cards)s)]
[parameters: [{'client_id': 825, 'current_age': 53, 'retirement_age': 66, 'birth_year': 1966, 'birth_month': 11, 'gender': 'Female', 'address': '462 Rose Lane', 'latitude': 34.15, 'longitude': -117.76, 'per_capita_income': 29278, 'yearly_income': 59696, 'total_debt': 127613, 'credit_score': 787, 'num_credit_cards': 5}, {'client_id': 1746, 'current_age': 53, 'retirement_age': 68, 'birth_year': 1966, 'birth_month': 12, 'gender': 'Female', 'addre

In [3]:
# POPULATE MCC_CODES TABLE
import json

# Load JSON data
with open('/home/bnguyen/Desktop/finance_analytics/datasets/mcc_codes.json') as f:
    mcc_dict = json.load(f)

df_mcc = pd.DataFrame(list(mcc_dict.items()), columns=['mcc', 'merchant_type'])
df_mcc['mcc'] = df_mcc['mcc'].astype(int)
df_mcc.to_sql('mcc_codes', con=engine, if_exists='append', index=False)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '5812' for key 'mcc_codes.PRIMARY'")
[SQL: INSERT INTO mcc_codes (mcc, merchant_type) VALUES (%(mcc)s, %(merchant_type)s)]
[parameters: [{'mcc': 5812, 'merchant_type': 'Eating Places and Restaurants'}, {'mcc': 5541, 'merchant_type': 'Service Stations'}, {'mcc': 7996, 'merchant_type': 'Amusement Parks, Carnivals, Circuses'}, {'mcc': 5411, 'merchant_type': 'Grocery Stores, Supermarkets'}, {'mcc': 4784, 'merchant_type': 'Tolls and Bridge Fees'}, {'mcc': 4900, 'merchant_type': 'Utilities - Electric, Gas, Water, Sanitary'}, {'mcc': 5942, 'merchant_type': 'Book Stores'}, {'mcc': 5814, 'merchant_type': 'Fast Food Restaurants'}  ... displaying 10 of 109 total bound parameter sets ...  {'mcc': 3009, 'merchant_type': 'Fabricated Structural Metal Products'}, {'mcc': 5733, 'merchant_type': 'Music Stores - Musical Instruments'}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [4]:
# POPULATE CARDS TABLE
df = pd.read_csv('/home/bnguyen/Desktop/finance_analytics/datasets/cards_data.csv')

# column rename
df.rename(columns={
    'id': 'card_id',
}, inplace=True)

# remove $, convert to numeric
df['credit_limit'] = pd.to_numeric(df['credit_limit'].astype(str).str.replace('$', ''), errors='coerce')

# convert date
df['expires'] = pd.to_datetime(df['expires'], format='%m/%Y', errors='coerce').dt.to_period('M').dt.to_timestamp().dt.date

# convert date
df['acct_open_date'] = pd.to_datetime(df['acct_open_date'], format='%m/%Y', errors='coerce')
df['acct_open_date'] = df['acct_open_date'].fillna(pd.to_datetime(df['acct_open_date'], format='%d/%m/%Y', errors='coerce'))
df['acct_open_date'] = df['acct_open_date'].dt.to_period('M').dt.to_timestamp().dt.date

df.to_sql('cards', con=engine, if_exists='append', index=False, chunksize=60000)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '4524' for key 'cards.PRIMARY'")
[SQL: INSERT INTO cards (card_id, client_id, card_brand, card_type, card_number, expires, cvv, has_chip, num_cards_issued, credit_limit, acct_open_date, year_pin_last_changed, card_on_dark_web) VALUES (%(card_id)s, %(client_id)s, %(card_brand)s, %(card_type)s, %(card_number)s, %(expires)s, %(cvv)s, %(has_chip)s, %(num_cards_issued)s, %(credit_limit)s, %(acct_open_date)s, %(year_pin_last_changed)s, %(card_on_dark_web)s)]
[parameters: [{'card_id': 4524, 'client_id': 825, 'card_brand': 'Visa', 'card_type': 'Debit', 'card_number': 4344676511950444, 'expires': datetime.date(2022, 12, 1), 'cvv': 623, 'has_chip': 'YES', 'num_cards_issued': 2, 'credit_limit': 24295, 'acct_open_date': datetime.date(2002, 9, 1), 'year_pin_last_changed': 2008, 'card_on_dark_web': 'No'}, {'card_id': 2731, 'client_id': 825, 'card_brand': 'Visa', 'card_type': 'Debit', 'card_number': 4956965974959986, 'expires': date

In [5]:
# POPULATE TRANSACTIONS TABLE

# # Truncate table before inserting
# with engine.connect() as conn:
#     conn.execute(text("TRUNCATE TABLE transactions"))
#     conn.commit()

df = pd.read_csv('/home/bnguyen/Desktop/finance_analytics/datasets/transactions_data.csv')

# column rename
df.rename(columns={
    'id': 'transaction_id',
    'date': 'trans_date'
}, inplace=True)

# remove $, convert to numeric
df['amount'] = pd.to_numeric(df['amount'].astype(str).str.replace('$', ''), errors='coerce')

# convert datetime
df['trans_date'] = pd.to_datetime(df['trans_date'], errors='coerce')

df.to_sql('transactions', con=engine, if_exists='append', index=False, chunksize=50000)

KeyboardInterrupt: 

In [ ]:
# POPULATE FRAUD_LABELS TABLE

# # Truncate table before inserting
# with engine.connect() as conn:
#     conn.execute(text("TRUNCATE TABLE fraud_labels"))
#     conn.commit()

import json
from pathlib import Path
import pandas as pd
from sqlalchemy.exc import IntegrityError

labels_path = Path('/home/bnguyen/Desktop/finance_analytics/datasets/train_fraud_labels.json')
with labels_path.open('r') as f:
    raw = json.load(f)

# collect all transaction_id->label pairs where label is 'Yes' or 'No'
labels = {}
def collect_labels(obj):
    if isinstance(obj, dict):
        for k, v in obj.items():
            if isinstance(v, str) and v in ('Yes', 'No'):
                labels[k] = v
            else:
                collect_labels(v)
    elif isinstance(obj, list):
        for item in obj:
            collect_labels(item)

collect_labels(raw)

df_labels = pd.DataFrame(list(labels.items()), columns=['transaction_id', 'label'])
df_labels['transaction_id'] = pd.to_numeric(df_labels['transaction_id'], errors='coerce')
df_labels = df_labels.dropna(subset=['transaction_id'])
df_labels['transaction_id'] = df_labels['transaction_id'].astype(int)
df_labels = df_labels[df_labels['label'].isin(['Yes','No'])].drop_duplicates(['transaction_id'])

# avoid inserting duplicates and enforce FK to transactions
try:
    existing = pd.read_sql('SELECT transaction_id FROM fraud_labels', con=engine)
    existing_ids = set(existing['transaction_id'].astype(int).tolist()) if not existing.empty else set()
except Exception:
    existing_ids = set()

try:
    tx = pd.read_sql('SELECT transaction_id FROM transactions', con=engine)
    valid_tx_ids = set(tx['transaction_id'].astype(int).tolist()) if not tx.empty else set()
except Exception:
    valid_tx_ids = set()

df_new = df_labels[~df_labels['transaction_id'].isin(existing_ids)]
if valid_tx_ids:
    df_new = df_new[df_new['transaction_id'].isin(valid_tx_ids)]

if df_new.empty:
    print("No new fraud labels to insert.")
else:
    try:
        df_new.to_sql('fraud_labels', con=engine, if_exists='append', index=False, chunksize=50000)
        print(f"Inserted {len(df_new)} new fraud label(s).")
    except IntegrityError as e:
        print("IntegrityError on insert:", e)

Inserted 8914963 new fraud label(s).
